# Introduction to Spark with GDELT Data

Note that this notebook is using a slightly older version of Spark and is in Python 2.

Below is my standard collection of imports for doing analytics with Spark. This is being loaded in a Dockerized Conda environment talking to Spark.

In [20]:
from __future__ import print_function

import time

import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import warnings # current version of seaborns generates warnings
warnings.filterwarnings("ignore")

import seaborn as sns
sns.set(style="white", color_codes=True)

from pyspark.sql import SparkSession

I don't know that you always need Hive support. A lot of times, I use custom UDFs that are in a Java jar and I need to register those functions with Spark SQL, and Hive support is needed for that. In any case, we need a Spark session and context and this is how we grab it.

In [21]:
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

## Load Data

This takes a while. Spark is running through the data and making sure it has a good idea of what the schema looks like. Depending on what your data format is, this can be faster or slower. Avro seems to be really fast, because the file headers give the schema without needing to read all the data. JSON is a bit slower, as Spark needs to read the data to obtain the schema.

In [4]:
path = '/opt/datadir/data/gdelt/events'

events_df = spark.read.json(path)

events_df.printSchema()

root
 |-- ActionGeo_ADM1Code: string (nullable = true)
 |-- ActionGeo_ADM2Code: string (nullable = true)
 |-- ActionGeo_CountryCode: string (nullable = true)
 |-- ActionGeo_FeatureID: string (nullable = true)
 |-- ActionGeo_FullName: string (nullable = true)
 |-- ActionGeo_Lat: double (nullable = true)
 |-- ActionGeo_Long: string (nullable = true)
 |-- ActionGeo_Type: long (nullable = true)
 |-- Actor1Code: string (nullable = true)
 |-- Actor1CountryCode: string (nullable = true)
 |-- Actor1EthnicCode: string (nullable = true)
 |-- Actor1Geo_ADM1Code: string (nullable = true)
 |-- Actor1Geo_ADM2Code: string (nullable = true)
 |-- Actor1Geo_CountryCode: string (nullable = true)
 |-- Actor1Geo_FeatureID: string (nullable = true)
 |-- Actor1Geo_FullName: string (nullable = true)
 |-- Actor1Geo_Lat: double (nullable = true)
 |-- Actor1Geo_Long: string (nullable = true)
 |-- Actor1Geo_Type: long (nullable = true)
 |-- Actor1KnownGroupCode: string (nullable = true)
 |-- Actor1Name: string (n

In [8]:
events_df.head(5)

[Row(ActionGeo_ADM1Code=u'CH04', ActionGeo_ADM2Code=u'13155', ActionGeo_CountryCode=u'CH', ActionGeo_FeatureID=u'-3865821', ActionGeo_FullName=u'Zhongyuan, Jiangsu, China', ActionGeo_Lat=32.3771, ActionGeo_Long=u'120.194', ActionGeo_Type=4, Actor1Code=u'CHN', Actor1CountryCode=u'CHN', Actor1EthnicCode=None, Actor1Geo_ADM1Code=u'CH04', Actor1Geo_ADM2Code=u'13155', Actor1Geo_CountryCode=u'CH', Actor1Geo_FeatureID=u'-3865821', Actor1Geo_FullName=u'Zhongyuan, Jiangsu, China', Actor1Geo_Lat=32.3771, Actor1Geo_Long=u'120.194', Actor1Geo_Type=4, Actor1KnownGroupCode=None, Actor1Name=u'CHINA', Actor1Religion1Code=None, Actor1Religion2Code=None, Actor1Type1Code=None, Actor1Type2Code=None, Actor1Type3Code=None, Actor2Code=None, Actor2CountryCode=None, Actor2EthnicCode=None, Actor2Geo_ADM1Code=None, Actor2Geo_ADM2Code=None, Actor2Geo_CountryCode=None, Actor2Geo_FeatureID=None, Actor2Geo_FullName=None, Actor2Geo_Lat=None, Actor2Geo_Long=None, Actor2Geo_Type=0, Actor2KnownGroupCode=None, Actor2Name

Make a temp table. This allows us to run SQL queries on the JSON data. Spark isn't doing much at this point.

In [10]:
events_df.createOrReplaceTempView('events')

In [12]:
query = """
select * from events limit 100
"""

results_df = spark.sql(query)

Pushing the Spark dataframe into a Pandas dataframe makes printing easier. In the imports above, you can see that we've told Pandas to display all columns.

In [13]:
results_df.toPandas().head()

,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_CountryCode,ActionGeo_FeatureID,ActionGeo_FullName,ActionGeo_Lat,ActionGeo_Long,ActionGeo_Type,Actor1Code,Actor1CountryCode,Actor1EthnicCode,Actor1Geo_ADM1Code,Actor1Geo_ADM2Code,Actor1Geo_CountryCode,Actor1Geo_FeatureID,Actor1Geo_FullName,Actor1Geo_Lat,Actor1Geo_Long,Actor1Geo_Type,Actor1KnownGroupCode,Actor1Name,Actor1Religion1Code,Actor1Religion2Code,Actor1Type1Code,Actor1Type2Code,Actor1Type3Code,Actor2Code,Actor2CountryCode,Actor2EthnicCode,Actor2Geo_ADM1Code,Actor2Geo_ADM2Code,Actor2Geo_CountryCode,Actor2Geo_FeatureID,Actor2Geo_FullName,Actor2Geo_Lat,Actor2Geo_Long,Actor2Geo_Type,Actor2KnownGroupCode,Actor2Name,Actor2Religion1Code,Actor2Religion2Code,Actor2Type1Code,Actor2Type2Code,Actor2Type3Code,AvgTone,CAMEOCodeDescription,DATEADDED,EventBaseCode,EventCode,EventRootCode,FractionDate,GLOBALEVENTID,GoldsteinScale,IsRootEvent,MonthYear,NumArticles,NumMentions,NumSources,QuadClass,SOURCEURL,SQLDATE,year,month,day
0,CH04,13155,CH,-3865821,"Zhongyuan, Jiangsu, China",32.3771,120.194,4,CHN,CHN,None,CH04,13155,CH,-3865821,"Zhongyuan, Jiangsu, China",32.3771,120.194,4,None,CHINA,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,0,None,None,None,None,None,None,None,-4.161412,Cooperate economically,20180520003000,061,061,06,2017.3836,757486222,6.4,1,201705,5,5,1,2,http://www.businessinsider.com/r-cutting-200-b...,20170520,2017,5,20
1,USVA,None,US,1760674,"Liberty University, Virginia, United States",37.3507,-79.1809,3,EDU,None,None,USVA,VA680,US,1760674,"Liberty University, Virginia, United States",37.3507,-79.1809,3,None,UNIVERSITY,None,None,EDU,None,None,GOV,None,None,USVA,None,US,1760674,"Liberty University, Virginia, United States",37.3507,-79.1809,3,None,PRESIDENT,None,None,GOV,None,None,5.412157,Host a visit,20180520003000,043,043,04,2017.3836,757486223,2.8,1,201705,10,10,1,1,http://www.ssuchronicle.com/2018/05/19/preside...,20170520,2017,5,20
2,USVA,None,US,1760674,"Liberty University, Virginia, United States",37.3507,-79.1809,3,GOV,None,None,USVA,VA680,US,1760674,"Liberty University, Virginia, United States",37.3507,-79.1809,3,None,PRESIDENT,None,None,GOV,None,None,EDU,None,None,USVA,None,US,1760674,"Liberty University, Virginia, United States",37.3507,-79.1809,3,None,UNIVERSITY,None,None,EDU,None,None,5.412157,Make a visit,20180520003000,042,042,04,2017.3836,757486224,1.9,1,201705,10,10,1,1,http://www.ssuchronicle.com/2018/05/19/preside...,20170520,2017,5,20
3,RP,None,RP,RP,Philippines,13.0000,122.0,1,PHL,PHL,None,RP,None,RP,RP,Philippines,13.0000,122.0,1,None,PHILIPPINE,None,None,None,None,None,CHR,None,None,RP,None,RP,RP,Philippines,13.0000,122.0,1,None,CHRISTIAN,CHR,None,None,None,None,0.269906,"Make statement, not specified below",20180520003000,010,010,01,2017.3836,757486225,0.0,0,201705,1,1,1,1,http://www.asianjournal.com/judge-overturns-ca...,20170520,2017,5,20
4,RP,None,RP,RP,Philippines,13.0000,122.0,1,PHL,PHL,None,RP,None,RP,RP,Philippines,13.0000,122.0,1,None,PHILIPPINE,None,None,None,None,None,CHR,None,None,RP,None,RP,RP,Philippines,13.0000,122.0,1,None,CHRISTIAN,CHR,None,None,None,None,0.269906,"Reject, not specified below",20180520003000,120,120,12,2017.3836,757486226,-4.0,0,201705,1,1,1,3,http://www.asianjournal.com/judge-overturns-ca...,20170520,2017,5,20


## ETL the Data

I want to get the hosts serving the events, chopping off any event data from the hostname. We can do that with some substring parsing in SQL. I'm also only going to grab the data that I want for my investigation.

In [18]:
query = """
select
    AvgTone as avg_tone,
    CAMEOCodeDescription as code_desc,
    EventCode as event_code,
    EventRootCode as event_root_code,
    IsRootEvent as is_root_event,
    NumArticles as num_articles,
    NumMentions as num_mentions,
    substring_index(substring_index(SOURCEURL, '//', -1), '/', 1) as source_url,
    year, month, day
from events
limit 1000
"""

results_df = spark.sql(query)

results_df.toPandas().head(100)

,avg_tone,code_desc,event_code,event_root_code,is_root_event,num_articles,num_mentions,source_url,year,month,day
0,-4.161412,Cooperate economically,061,06,1,5,5,www.businessinsider.com,2017,5,20
1,5.412157,Host a visit,043,04,1,10,10,www.ssuchronicle.com,2017,5,20
2,5.412157,Make a visit,042,04,1,10,10,www.ssuchronicle.com,2017,5,20
3,0.269906,"Make statement, not specified below",010,01,0,1,1,www.asianjournal.com,2017,5,20
4,0.269906,"Reject, not specified below",120,12,0,1,1,www.asianjournal.com,2017,5,20
5,-2.318487,"Consult, not specified below",040,04,1,4,4,www.orlandosentinel.com,2017,5,20
6,0.269906,"Make statement, not specified below",010,01,0,3,3,www.asianjournal.com,2017,5,20
7,0.269906,"Reject, not specified below",120,12,0,3,3,www.asianjournal.com,2017,5,20
8,0.269906,"Make statement, not specified below",010,01,0,4,4,www.asianjournal.com,2017,5,20
9,0.269906,"Reject, not specified below",120,12,0,4,4,www.asianjournal.com,2017,5,20


We're going to save the data in Parquet format. This is a columnar format that makes specific queries (especially analytics queries) far faster. It also makes some queries slower. When we save, Spark will automatically snappy compress the files.

In [19]:
query = """
select
    AvgTone as avg_tone,
    CAMEOCodeDescription as code_desc,
    EventCode as event_code,
    EventRootCode as event_root_code,
    IsRootEvent as is_root_event,
    NumArticles as num_articles,
    NumMentions as num_mentions,
    substring_index(substring_index(SOURCEURL, '//', -1), '/', 1) as source_url,
    year, month, day
from events
"""

filepath_parquet = "/opt/datadir/data/gdelt_etl/tone_and_source/"

spark.sql(query).write.parquet(filepath_parquet, mode="overwrite")

## Load the Saved Data and Query

We've gone from having 220GB of data in JSON format to having around 1.4GB. This will make our queries far faster.

In [22]:
filepath_parquet = "/opt/datadir/data/gdelt_etl/tone_and_source/"

df_parquet = spark.read.parquet(filepath_parquet)

df_parquet.createOrReplaceTempView('events_parquet')

Some queries are very fast. Because we're looking at columnar data, counts are very fast.

In [7]:
import time

query = """
select count(*) as num from events_parquet
"""

start_time = time.time()
spark.sql(query).show()
elapsed_time = time.time() - start_time
print(elapsed_time)

+---------+
|      num|
+---------+
|146010251|
+---------+

0.750168085098


Spark can do exact percentile values, and these can take some time. There is also an approximate percentile function which runs far faster. Here, we'll use the exact method.

In [33]:
query = """
select
    year, month,
    count(*) as events,
    avg(avg_tone) as mean_tone,
    percentile(avg_tone, 0.1) as p10_tone,
    percentile(avg_tone, 0.25) as p25_tone,
    percentile(avg_tone, 0.5) as p50_tone,
    percentile(avg_tone, 0.75) as p75_tone,
    percentile(avg_tone, 0.9) as p90_tone
from events_parquet
where year = 2018
group by year, month
order by year, month
"""

start_time = time.time()
dfp = spark.sql(query).toPandas()
elapsed_time = time.time() - start_time
print(elapsed_time)

23.0223920345


In [34]:
dfp.head(100)

,year,month,events,mean_tone,p10_tone,p25_tone,p50_tone,p75_tone,p90_tone
0,2018,1,5051092,-2.078976,-7.106599,-4.485050,-1.846591,0.552486,2.578269
1,2018,2,4837577,-2.079025,-7.152875,-4.591440,-1.875000,0.619195,2.694611
2,2018,3,5585566,-1.909961,-6.868687,-4.304636,-1.709402,0.684932,2.759740
3,2018,4,5361310,-2.042566,-7.025761,-4.516129,-1.851852,0.595948,2.673147
4,2018,5,5518302,-1.843049,-6.809339,-4.211957,-1.639344,0.740741,2.823529
5,2018,6,5039461,-1.834840,-6.727829,-4.155844,-1.624884,0.704225,2.748414
6,2018,7,4990580,-1.995033,-6.923077,-4.388489,-1.814882,0.594354,2.685185
7,2018,8,5139643,-2.154502,-7.151442,-4.674221,-2.005420,0.546448,2.631579
8,2018,9,4916944,-2.040082,-7.047872,-4.552469,-1.898734,0.672453,2.790698
9,2018,10,5323981,-1.892544,-6.825939,-4.310345,-1.757632,0.727273,2.843602


Let's take a list of known sites that are unreputable or highly divisive. We're going to have to use a case block which was formed in a text editor to search for instances of these sites, since we aren't sure how they'll always show up in the data.

In [10]:
query = """
select
    sq0.year as year,
    sq0.month as month,
    count(*) as num,
    sum(sq0.not_reputable) as not_reputable
    from (
        select
        CASE
            WHEN source_url LIKE "%nationalreport.net%" THEN 1
            WHEN source_url LIKE "%huzlers.com%" THEN 1
            WHEN source_url LIKE "%infowars.com%" THEN 1
            WHEN source_url LIKE "%theonion.com%" THEN 1
            WHEN source_url LIKE "%empirenews.net%" THEN 1
            WHEN source_url LIKE "%activistpost.com%" THEN 1
            WHEN source_url LIKE "%americannews.com%" THEN 1
            WHEN source_url LIKE "%christwire.org%" THEN 1
            WHEN source_url LIKE "%civictribune.com%" THEN 1
            WHEN source_url LIKE "%duffelblog.com%" THEN 1
            WHEN source_url LIKE "%en.mediamass.net%" THEN 1
            WHEN source_url LIKE "%naturalnews.com%" THEN 1
            WHEN source_url LIKE "%realnewsrightnow.com%" THEN 1
            WHEN source_url LIKE "%rilenews.com%" THEN 1
            WHEN source_url LIKE "%thedcgazette.com%" THEN 1
            WHEN source_url LIKE "%worldnewsdailyreport.com%" THEN 1
            WHEN source_url LIKE "%21stcenturywire.com%" THEN 1
            WHEN source_url LIKE "%70news.wordpress.com%" THEN 1
            WHEN source_url LIKE "%beforeitsnews.com%" THEN 1
            WHEN source_url LIKE "%breitbart.com%" THEN 1
            WHEN source_url LIKE "%clickhole.com%" THEN 1
            WHEN source_url LIKE "%coasttocoastam.com%" THEN 1
            WHEN source_url LIKE "%consciouslifenews.com%" THEN 1
            WHEN source_url LIKE "%conservativeoutfitters.com%" THEN 1
            WHEN source_url LIKE "%countdowntozerotime.com%" THEN 1
            WHEN source_url LIKE "%creambmp.com%" THEN 1
            WHEN source_url LIKE "%dailybuzzlive.com%" THEN 1
            WHEN source_url LIKE "%disclose.tv%" THEN 1
            WHEN source_url LIKE "%empiresports.co%" THEN 1
            WHEN source_url LIKE "%globalresearch.ca%" THEN 1
            WHEN source_url LIKE "%newslo.com%" THEN 1
            WHEN source_url LIKE "%newsmutiny.com%" THEN 1
            WHEN source_url LIKE "%veteranstoday.com%" THEN 1
            WHEN source_url LIKE "%worldtruth.tv%" THEN 1
            WHEN source_url LIKE "%abcnews.com.co%" THEN 1
            WHEN source_url LIKE "%addictinginfo.org%" THEN 1
            WHEN source_url LIKE "%bipartisanreport.com%" THEN 1
            WHEN source_url LIKE "%bluenationreview.com%" THEN 1
            WHEN source_url LIKE "%chronicle.su%" THEN 1
            WHEN source_url LIKE "%counterpsyops.com%" THEN 1
            WHEN source_url LIKE "%dailycurrant.com%" THEN 1
            WHEN source_url LIKE "%dcclothesline.com%" THEN 1
            WHEN source_url LIKE "%duhprogressive.com%" THEN 1
            WHEN source_url LIKE "%enduringvision.com%" THEN 1
            WHEN source_url LIKE "%fprnradio.com%" THEN 1
            WHEN source_url LIKE "%geoengineeringwatch.org%" THEN 1
            WHEN source_url LIKE "%govtslaves.info%" THEN 1
            WHEN source_url LIKE "%gulagbound.com%" THEN 1
            WHEN source_url LIKE "%hangthebankers.com%" THEN 1
            WHEN source_url LIKE "%intellihub.com%" THEN 1
            WHEN source_url LIKE "%jonesreport.com%" THEN 1
            WHEN source_url LIKE "%lewrockwell.com%" THEN 1
            WHEN source_url LIKE "%libertymovementradio.com%" THEN 1
            WHEN source_url LIKE "%libertytalk.fm%" THEN 1
            WHEN source_url LIKE "%newsbiscuit.com%" THEN 1
            WHEN source_url LIKE "%newswire-24.com%" THEN 1
            WHEN source_url LIKE "%nodisinfo.com%" THEN 1
            WHEN source_url LIKE "%now8news.com%" THEN 1
            WHEN source_url LIKE "%nowtheendbegins.com%" THEN 1
            WHEN source_url LIKE "%pakalertpress.com%" THEN 1
            WHEN source_url LIKE "%politicops.com%" THEN 1
            WHEN source_url LIKE "%prisonplanet.com%" THEN 1
            WHEN source_url LIKE "%react365.com%" THEN 1
            WHEN source_url LIKE "%redflagnews.com%" THEN 1
            WHEN source_url LIKE "%thedailysheeple.com%" THEN 1
            WHEN source_url LIKE "%thenewsnerd.com%" THEN 1
            WHEN source_url LIKE "%therundownlive.com%" THEN 1
            WHEN source_url LIKE "%truthfrequencyradio.com%" THEN 1
            WHEN source_url LIKE "%USAToday.com.co%" THEN 1
            WHEN source_url LIKE "%weeklyworldnews.com%" THEN 1
            WHEN source_url LIKE "%witscience.org%" THEN 1
            WHEN source_url LIKE "%wnd.com%" THEN 1
            WHEN source_url LIKE "%anonews.co%" THEN 1
            WHEN source_url LIKE "%bigamericannews.com%" THEN 1
            WHEN source_url LIKE "%callthecops.net%" THEN 1
            WHEN source_url LIKE "%collective-evolution.com%" THEN 1
            WHEN source_url LIKE "%dailycaller.com%" THEN 1
            WHEN source_url LIKE "%denverguardian.com%" THEN 1
            WHEN source_url LIKE "%derfmagazine.com%" THEN 1
            WHEN source_url LIKE "%empireherald.com%" THEN 1
            WHEN source_url LIKE "%endingthefed.com%" THEN 1
            WHEN source_url LIKE "%globalassociatednews.com%" THEN 1
            WHEN source_url LIKE "%humansarefree.com%" THEN 1
            WHEN source_url LIKE "%ifyouonlynews.com%" THEN 1
            WHEN source_url LIKE "%itaglive.com%" THEN 1
            WHEN source_url LIKE "%libertyvideos.org%" THEN 1
            WHEN source_url LIKE "%madworldnews.com%" THEN 1
            WHEN source_url LIKE "%ncscooper.com%" THEN 1
            WHEN source_url LIKE "%newsexaminer.com%" THEN 1
            WHEN source_url LIKE "%occupydemocrats.com%" THEN 1
            WHEN source_url LIKE "%other98.com%" THEN 1
            WHEN source_url LIKE "%politicalblindspot.com%" THEN 1
            WHEN source_url LIKE "%politicalears.com%" THEN 1
            WHEN source_url LIKE "%realfarmacy.com%" THEN 1
            WHEN source_url LIKE "%redstate.com%" THEN 1
            WHEN source_url LIKE "%rt.com%" THEN 1
            WHEN source_url LIKE "%satiratribune.com%" THEN 1
            WHEN source_url LIKE "%sputniknews.com%" THEN 1
            WHEN source_url LIKE "%thebostontribune.com%" THEN 1
            WHEN source_url LIKE "%thefreethoughtproject.com%" THEN 1
            WHEN source_url LIKE "%thelapine.ca%" THEN 1
            WHEN source_url LIKE "%thespoof.com%" THEN 1
            WHEN source_url LIKE "%theuspatriot.com%" THEN 1
            WHEN source_url LIKE "%truepundit.com%" THEN 1
            WHEN source_url LIKE "%twitchy.com%" THEN 1
            WHEN source_url LIKE "%unconfirmedsources.com%" THEN 1
            WHEN source_url LIKE "%usuncut.com%" THEN 1
            WHEN source_url LIKE "%wakingupwisconsin.com%" THEN 1
            WHEN source_url LIKE "%yournewswire.com" THEN 1
            ELSE 0
        END as not_reputable,
        month,
        year
    from events_parquet
    where year >= 2017 and year <= 2019
    ) sq0
    group by sq0.year, sq0.month
    order by sq0.year, sq0.month
"""

start_time = time.time()
dfp = spark.sql(query).toPandas()
elapsed_time = time.time() - start_time
print(elapsed_time)

99.4048769474


In [11]:
dfp.head(100)

,year,month,num,not_reputable
0,2017,1,6144894,72636
1,2017,2,6055968,77292
2,2017,3,6368174,76780
3,2017,4,5615147,66986
4,2017,5,5801996,66590
5,2017,6,5401707,61799
6,2017,7,5046430,58445
7,2017,8,5307354,62625
8,2017,9,5237471,53287
9,2017,10,5323151,53672


Now, we'll use some analytics queries to see how these sites are producing material month over month. We'll rank each sites months by the events they are creating and see how those months compare to the previous month in the results. We'll turn this into a Pandas dataframe and export the results as a CSV.

In [33]:
query = """
select
    sq1.source_url as source_url,
    sq1.events as events,
    sq1.events - (lag(sq1.events) over (partition by sq1.source_url order by sq1.year, sq1.month)) as events_diff,
    rank() over (partition by sq1.source_url order by sq1.events desc) as source_month_rank,
    sq1.year as year,
    sq1.month as month
from (
    select
        sq0.source_url as source_url,
        sum(sq0.not_reputable) as events,
        sq0.year as year,
        sq0.month as month,
        sq0.not_reputable
    from (
        select
            source_url,
            CASE
                WHEN source_url LIKE "%nationalreport.net%" THEN 1
                WHEN source_url LIKE "%huzlers.com%" THEN 1
                WHEN source_url LIKE "%infowars.com%" THEN 1
                WHEN source_url LIKE "%theonion.com%" THEN 1
                WHEN source_url LIKE "%empirenews.net%" THEN 1
                WHEN source_url LIKE "%activistpost.com%" THEN 1
                WHEN source_url LIKE "%americannews.com%" THEN 1
                WHEN source_url LIKE "%christwire.org%" THEN 1
                WHEN source_url LIKE "%civictribune.com%" THEN 1
                WHEN source_url LIKE "%duffelblog.com%" THEN 1
                WHEN source_url LIKE "%en.mediamass.net%" THEN 1
                WHEN source_url LIKE "%naturalnews.com%" THEN 1
                WHEN source_url LIKE "%realnewsrightnow.com%" THEN 1
                WHEN source_url LIKE "%rilenews.com%" THEN 1
                WHEN source_url LIKE "%thedcgazette.com%" THEN 1
                WHEN source_url LIKE "%worldnewsdailyreport.com%" THEN 1
                WHEN source_url LIKE "%21stcenturywire.com%" THEN 1
                WHEN source_url LIKE "%70news.wordpress.com%" THEN 1
                WHEN source_url LIKE "%beforeitsnews.com%" THEN 1
                WHEN source_url LIKE "%breitbart.com%" THEN 1
                WHEN source_url LIKE "%clickhole.com%" THEN 1
                WHEN source_url LIKE "%coasttocoastam.com%" THEN 1
                WHEN source_url LIKE "%consciouslifenews.com%" THEN 1
                WHEN source_url LIKE "%conservativeoutfitters.com%" THEN 1
                WHEN source_url LIKE "%countdowntozerotime.com%" THEN 1
                WHEN source_url LIKE "%creambmp.com%" THEN 1
                WHEN source_url LIKE "%dailybuzzlive.com%" THEN 1
                WHEN source_url LIKE "%disclose.tv%" THEN 1
                WHEN source_url LIKE "%empiresports.co%" THEN 1
                WHEN source_url LIKE "%globalresearch.ca%" THEN 1
                WHEN source_url LIKE "%newslo.com%" THEN 1
                WHEN source_url LIKE "%newsmutiny.com%" THEN 1
                WHEN source_url LIKE "%veteranstoday.com%" THEN 1
                WHEN source_url LIKE "%worldtruth.tv%" THEN 1
                WHEN source_url LIKE "%abcnews.com.co%" THEN 1
                WHEN source_url LIKE "%addictinginfo.org%" THEN 1
                WHEN source_url LIKE "%bipartisanreport.com%" THEN 1
                WHEN source_url LIKE "%bluenationreview.com%" THEN 1
                WHEN source_url LIKE "%chronicle.su%" THEN 1
                WHEN source_url LIKE "%counterpsyops.com%" THEN 1
                WHEN source_url LIKE "%dailycurrant.com%" THEN 1
                WHEN source_url LIKE "%dcclothesline.com%" THEN 1
                WHEN source_url LIKE "%duhprogressive.com%" THEN 1
                WHEN source_url LIKE "%enduringvision.com%" THEN 1
                WHEN source_url LIKE "%fprnradio.com%" THEN 1
                WHEN source_url LIKE "%geoengineeringwatch.org%" THEN 1
                WHEN source_url LIKE "%govtslaves.info%" THEN 1
                WHEN source_url LIKE "%gulagbound.com%" THEN 1
                WHEN source_url LIKE "%hangthebankers.com%" THEN 1
                WHEN source_url LIKE "%intellihub.com%" THEN 1
                WHEN source_url LIKE "%jonesreport.com%" THEN 1
                WHEN source_url LIKE "%lewrockwell.com%" THEN 1
                WHEN source_url LIKE "%libertymovementradio.com%" THEN 1
                WHEN source_url LIKE "%libertytalk.fm%" THEN 1
                WHEN source_url LIKE "%newsbiscuit.com%" THEN 1
                WHEN source_url LIKE "%newswire-24.com%" THEN 1
                WHEN source_url LIKE "%nodisinfo.com%" THEN 1
                WHEN source_url LIKE "%now8news.com%" THEN 1
                WHEN source_url LIKE "%nowtheendbegins.com%" THEN 1
                WHEN source_url LIKE "%pakalertpress.com%" THEN 1
                WHEN source_url LIKE "%politicops.com%" THEN 1
                WHEN source_url LIKE "%prisonplanet.com%" THEN 1
                WHEN source_url LIKE "%react365.com%" THEN 1
                WHEN source_url LIKE "%redflagnews.com%" THEN 1
                WHEN source_url LIKE "%thedailysheeple.com%" THEN 1
                WHEN source_url LIKE "%thenewsnerd.com%" THEN 1
                WHEN source_url LIKE "%therundownlive.com%" THEN 1
                WHEN source_url LIKE "%truthfrequencyradio.com%" THEN 1
                WHEN source_url LIKE "%USAToday.com.co%" THEN 1
                WHEN source_url LIKE "%weeklyworldnews.com%" THEN 1
                WHEN source_url LIKE "%witscience.org%" THEN 1
                WHEN source_url LIKE "%wnd.com%" THEN 1
                WHEN source_url LIKE "%anonews.co%" THEN 1
                WHEN source_url LIKE "%bigamericannews.com%" THEN 1
                WHEN source_url LIKE "%callthecops.net%" THEN 1
                WHEN source_url LIKE "%collective-evolution.com%" THEN 1
                WHEN source_url LIKE "%dailycaller.com%" THEN 1
                WHEN source_url LIKE "%denverguardian.com%" THEN 1
                WHEN source_url LIKE "%derfmagazine.com%" THEN 1
                WHEN source_url LIKE "%empireherald.com%" THEN 1
                WHEN source_url LIKE "%endingthefed.com%" THEN 1
                WHEN source_url LIKE "%globalassociatednews.com%" THEN 1
                WHEN source_url LIKE "%humansarefree.com%" THEN 1
                WHEN source_url LIKE "%ifyouonlynews.com%" THEN 1
                WHEN source_url LIKE "%itaglive.com%" THEN 1
                WHEN source_url LIKE "%libertyvideos.org%" THEN 1
                WHEN source_url LIKE "%madworldnews.com%" THEN 1
                WHEN source_url LIKE "%ncscooper.com%" THEN 1
                WHEN source_url LIKE "%newsexaminer.com%" THEN 1
                WHEN source_url LIKE "%occupydemocrats.com%" THEN 1
                WHEN source_url LIKE "%other98.com%" THEN 1
                WHEN source_url LIKE "%politicalblindspot.com%" THEN 1
                WHEN source_url LIKE "%politicalears.com%" THEN 1
                WHEN source_url LIKE "%realfarmacy.com%" THEN 1
                WHEN source_url LIKE "%redstate.com%" THEN 1
                WHEN source_url LIKE "%.rt.com%" THEN 1
                WHEN source_url LIKE "%satiratribune.com%" THEN 1
                WHEN source_url LIKE "%sputniknews.com%" THEN 1
                WHEN source_url LIKE "%thebostontribune.com%" THEN 1
                WHEN source_url LIKE "%thefreethoughtproject.com%" THEN 1
                WHEN source_url LIKE "%thelapine.ca%" THEN 1
                WHEN source_url LIKE "%thespoof.com%" THEN 1
                WHEN source_url LIKE "%theuspatriot.com%" THEN 1
                WHEN source_url LIKE "%truepundit.com%" THEN 1
                WHEN source_url LIKE "%twitchy.com%" THEN 1
                WHEN source_url LIKE "%unconfirmedsources.com%" THEN 1
                WHEN source_url LIKE "%usuncut.com%" THEN 1
                WHEN source_url LIKE "%wakingupwisconsin.com%" THEN 1
                WHEN source_url LIKE "%yournewswire.com" THEN 1
                ELSE 0
            END as not_reputable,
            month,
            year
        from events_parquet
        where year >= 2017 and year <= 2019
        ) sq0
    where sq0.not_reputable=1
    group by
        sq0.source_url, sq0.not_reputable, sq0.year, sq0.month
    order by
        sq0.source_url, sq0.not_reputable, sq0.year, sq0.month
) sq1
order by sq1.source_url, sq1.year, sq1.month
"""

start_time = time.time()
dfp = spark.sql(query).toPandas()
elapsed_time = time.time() - start_time
print(elapsed_time)

93.8160510063


In [34]:
dfp.head()

,source_url,events,events_diff,source_month_rank,year,month
0,21stcenturywire.com,414,NaN,7,2017,1
1,21stcenturywire.com,552,138.0,1,2017,2
2,21stcenturywire.com,460,-92.0,4,2017,3
3,21stcenturywire.com,364,-96.0,12,2017,4
4,21stcenturywire.com,529,165.0,2,2017,5


In [35]:
dfp.size

5388

In [36]:
dfp.to_csv('unreputable_source_month_rank.csv')